In [1]:
!which python

/Users/apabook/Desktop/ds-deployment/venv/bin/python


In [2]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete business flow

## Consts


In [3]:
data_space_business = "http://127.0.0.1:1206"
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
policy_id = ""
target_id = ""
template_id = ""
template = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding (in IT system)

In [4]:
url = data_space_provider + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
    "participant_slug": "Myself",
    "participant_type": "Provider",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-06-27T11:50:40.051487",
    "last_interaction": "2025-06-27T11:50:40.051492",
    "is_me": true
}


In [5]:
url = data_space_consumer + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
    "participant_slug": "Myself",
    "participant_type": "Consumer",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-06-27T11:50:32.564619",
    "last_interaction": "2025-06-27T11:50:32.564643",
    "is_me": true
}


In [6]:
url = data_space_consumer + "/api/v1/auth/manual/ssi"
payload = {
    "url": data_space_provider + "/api/v1/access",
    "id": "friendly_slugname_for_provider",
    "slug": "friendly_slugname_for_provider",
    "actions": "talk"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2FZVRg0EqNHGVK&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fpd%2FZVRg0EqNHGVK&client_id_scheme=redirect_uri&nonce=oZGSfZ4r6Ji2&response_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2FZVRg0EqNHGVK'

### Client logs in business via vp-presentation

In [7]:
url = data_space_business + "/gateway/api/login"
payload = {
    "authRequestId": "asdasdasdasdasd",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2Fasdasdasdasdasd&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fpd%2Fasdasdasdasdasd&client_id_scheme=redirect_uri&nonce=JXh8Nu57rlxM&response_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fapi%2Fv1%2Fverify%2Fasdasdasdasdasd'

### Provider and Consumer ids

In [8]:
url = data_space_provider + "/api/v1/busmates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json[1]["participant_id"]
data_space_consumer_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "id": "YyV0mTYon1PC",
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "token": "Sls10kwU9grqsVgdFIO7nTsDg8-hocGabpFfW9HoSBM",
        "token_actions": "talk",
        "saved_at": "2025-06-27T11:29:38.654827",
        "last_interaction": "2025-06-27T11:29:38.654830"
    },
    {
        "id": "NRx4oNK7IKQ1",
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "token": "dTh_I_YMnaxhCTGBOm5II6Bgn_YAgQJHIYB6EfOezjg",
        "token_actions": "talk",
        "saved_at": "2025-06-27T11:29:55.189287",
        "last_interaction": "2025-06-27T11:29:55.189290"
    },
    {
        "id": "L8M61dw7HEHN",
        "participant_id": "did:jwk:eyJrdHk

In [9]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json[1]["participant_id"]
data_space_consumer_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "participant_slug": "\"rainbow_consumer\"",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": "qHrQZ65nLs2fIPb9-pDZisQj0VWfuJ6UEq3folG-QLw",
        "token_actions": "talk",
        "saved_at": "2025-06-27T11:29:01.267481",
        "last_interaction": "2025-06-27T11:29:01.267482",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "participant_slug": "Myself",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": null,
        "token_actions": null,
        "sa

In [10]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoiWXU4b3dhN3pHZ191Vi1XN3ZpM0JVMnA1aExPTGNCZDNMQkRNXzY4ZUR4QSIsIngiOiIxSzFWc1lrZ3RmREh3WHJfUEF5X25FeFh1ZmIxT3NLQjloYXB4aEkwcl80In0",
        "participant_slug": "provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": "qHrQZ65nLs2fIPb9-pDZisQj0VWfuJ6UEq3folG-QLw",
        "token_actions": "talk",
        "saved_at": "2025-06-27T11:29:01.294330",
        "last_interaction": "2025-06-27T11:29:01.294331",
        "is_me": false
    },
    {
        "participant_id": "did:jwk:eyJrdHkiOiJPS1AiLCJjcnYiOiJFZDI1NTE5Iiwia2lkIjoibWlwSXF5TXpYLUpaQ3B6dnFmOFZTWDB5RlhjWjlqWDdMcHI5bDZVUGx0VSIsIngiOiJFX19lMXZIMm9rMDBUSEVTTkFIUXpNNl85b2pmWXJscGo2MzROZVBTdU9VIn0",
        "participant_slug": "Myself",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": null,
        "token_actions": null,
        "saved_at": "20

## Business User Pathway

### Fetching catalogs and datasets in Datahub

In [11]:
url = data_space_business + "/gateway/api/catalogs"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json[0]["urn"]
catalog_id

'urn:li:domain:medicamentos'

In [12]:
url = data_space_business + "/gateway/api/catalogs/{}/datasets".format(catalog_id)
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
dataset_id = response_as_json[0]["urn"]
dataset_id

'urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)'

In [13]:
url = data_space_business + "/gateway/api/catalogs/datasets/{}".format(dataset_id)
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "urn": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)",
    "name": "ACETAMINOPHEN_events",
    "platform": {
        "name": "airflow"
    },
    "description": "Informaci\u00f3n de los eventos producidos por el medicamento ACETAMINOPHEN",
    "tag_names": [
        "f\u00e1rmacos"
    ],
    "custom_properties": [
        [
            "identifier",
            "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
        ],
        [
            "creator",
            "US-FDA"
        ],
        [
            "keywords",
            "f\u00e1rmacos"
        ],
        [
            "access_url",
            "https://api.fda.gov/drug/event.json?search=patient.drug.medicinalproduct:ACETAMINOPHEN"
        ],
        [
            "landing_page",
            "https://138.4.7.113:9005"
        ],
        [
            "description",
            "Informaci\u00f3n de los eventos producidos por el medicamento ACETAMINOPHEN"
        ],
       

### Fetch policy templates

In [14]:
url = data_space_business + "/gateway/api/policy-templates"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
template_id = response_as_json[0]["id"]
template_id

IndexError: list index out of range

In [16]:
url = data_space_business + "/gateway/api/policy-templates/{}".format(template_id)
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
template = response_as_json
print(json.dumps(template, indent=4))


{
    "id": "template_1750925461",
    "title": "Advanced Content License Template",
    "description": "Template for defining usage permissions with multiple user-configurable conditions.",
    "content": {
        "obligation": null,
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "leftOperand": "spatial",
                        "operator": "eq",
                        "rightOperand": "$location"
                    }
                ]
            }
        ],
        "prohibition": null
    },
    "operand_options": {
        "$expiry_date": {
            "dataType": "xsd:dateTime",
            "defaultValue": "2026-12-31T23:59:59Z",
            "formType": "datetime",
            "label": [
                {
                    "@language": "es",
                    "@value": "Fecha de Expiraci\u00f3n"
                },
                {
                    "@language": "en",
   

### Create Policy from template

In [17]:
import ast

# select option
location_options = template["operand_options"]["$location"]["options"]
_location_options_inner = list(map(lambda option: option["value"], location_options))
location_choice = _location_options_inner[0]

# interpolate choice in content template
py = str(template["content"]).replace("$location", location_choice)
a = python_dict = ast.literal_eval(py)
json_string = json.dumps(a)
offer = json.loads(json_string)
print(json.dumps(offer, indent=4))



{
    "obligation": null,
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "leftOperand": "spatial",
                    "operator": "eq",
                    "rightOperand": "EU"
                }
            ]
        }
    ],
    "prohibition": null
}


In [18]:
url = data_space_business + "/gateway/api/catalogs/{}/datasets/{}/policies".format(catalog_id, dataset_id)
payload = offer
response = requests.request("POST", url, data=json.dumps(payload), headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:e6df048f-b238-4fb7-a010-ec3c29ca9e91",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "EU",
                    "leftOperand": "spatial",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": null,
    "@type": "Offer",
    "prohibition": null,
    "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
}


## Customer Pathway

### Customer fetches catalogs, datasets and offers

In [19]:
url = data_space_business + "/gateway/api/catalogs"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
catalog_id = response_as_json[0]["urn"]
catalog_id

'urn:li:domain:medicamentos'

In [20]:
url = data_space_business + "/gateway/api/catalogs/{}/datasets".format(catalog_id)
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
dataset_id = response_as_json[0]["urn"]
dataset_id

'urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)'

In [21]:
url = data_space_business + "/gateway/api/catalogs/datasets/{}".format(dataset_id)
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "urn": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)",
    "name": "ACETAMINOPHEN_events",
    "platform": {
        "name": "airflow"
    },
    "description": "Informaci\u00f3n de los eventos producidos por el medicamento ACETAMINOPHEN",
    "tag_names": [
        "f\u00e1rmacos"
    ],
    "custom_properties": [
        [
            "identifier",
            "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
        ],
        [
            "creator",
            "US-FDA"
        ],
        [
            "keywords",
            "f\u00e1rmacos"
        ],
        [
            "access_url",
            "https://api.fda.gov/drug/event.json?search=patient.drug.medicinalproduct:ACETAMINOPHEN"
        ],
        [
            "description",
            "Informaci\u00f3n de los eventos producidos por el medicamento ACETAMINOPHEN"
        ],
        [
            "landing_page",
            "https://138.4.7.113:9005"
        ],
       

In [22]:
url = data_space_business + "/gateway/api/catalogs/{}/datasets/{}/policies".format(catalog_id, dataset_id)
payload = offer
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))
offer = response_as_json[0]
print(json.dumps(offer, indent=4))

[
    {
        "@id": "urn:uuid:72832b8a-81ce-4c90-b553-2e884d53dc42",
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "EU",
                        "leftOperand": "spatial",
                        "operator": "eq"
                    }
                ]
            }
        ],
        "obligation": null,
        "@type": "Offer",
        "prohibition": null,
        "target": "urn:li:dataset:(urn:li:dataPlatform:airflow,ACETAMINOPHEN_events,PROD)"
    },
    {
        "@id": "urn:uuid:703fa341-a8ae-4524-80b9-0ed921d42a1d",
        "permission": [
            {
                "action": "use",
                "constraint": [
                    {
                        "rightOperand": "user",
                        "leftOperand": "did:web:hola.es",
                        "operator": "eq"
                    }
                ]
            }
        ],
        "obli

### Customer requests access to dataset under policy

In [23]:
url = data_space_business + "/gateway/api/negotiation/rpc/request"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "offer": {
        "@id": offer["@id"],
    }
}
response = requests.request("POST", url, data=json.dumps(payload), headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "error": "Failed to deserialize the JSON body into the target type: consumerParticipantId: invalid urn scheme at line 1 column 211"
}


### Business approves request and negotiation is done

In [ ]:
url = data_space_business + "/gateway/api/negotiation/rpc/accept"
payload = {
    "consumerPid": "urn:uuid:c1bf366a-f468-4848-a201-1a8124c970fb",
    "providerPid": "urn:uuid:a0967477-97e3-4046-9cca-a81c8112430e"
}